In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from utils import draw_lines

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

figsize = (8, 6)
save_figures = False

In [ ]:
#path = '/scratch/ssd/cciw/Videos-20200114T160100Z-001/Quad1'
#file = 'GLNI_12-1_2016-07-11_video-1.mp4'
file = 'GLNI_456-3_2015-07-17_video-1.mp4'
#outpath = file.split('.')[0]
#f = os.path.join(path, file)

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'

# Search for all video files on Google Drive...
all_videos = glob(os.path.join(DATA_PATH, 'Videos_and_stills/GLNI/*/*/*/Videos/Quad*/*.mp4'))
videotable_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
video_df = pd.read_csv(videotable_path, index_col=0)

vpath = video_df.iloc[video_df[video_df['Name'] == file].index]['Quadrat Video Path']
tokens = video_df[video_df['Name'] == file]['Quadrat Video Path'].values[0].split('\\')
#print(tokens)

video_path = DATA_PATH + '/Videos_and_stills/GLNI'
for tok in tokens[4:-1]:
    video_path += '/' + tok
video_path = os.path.join(video_path, file)
print('Loading video: ', video_path)

'''
out_path = os.path.join(video_path, 'data')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Making dir ', out_path)
'''

In [ ]:
# @param rho Distance resolution of the accumulator (pixels).
rho = 1  
# @param theta Angle resolution of the accumulator (radians).
theta = np.pi / 45

In [ ]:
cap = cv2.VideoCapture(video_path) #cap.set(cv2.CAP_PROP_FPS, FPS)
sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
      int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print(sz)

In [ ]:
# read first frame
ret, im = cap.read() 

if sz[0] == 1440:
    x_trim, y_trim = 1, 145
    canny_thresh1, canny_thresh2 = 60, 300
    """@param threshold Accumulator threshold, return 
    lines with more than threshold of votes. (intersection points)"""
    threshold = 125
    """@param minLineLength Minimum line length. 
    Line segments shorter than that are rejected. (pixels)"""
    mLL = 400
    """@param maxLineGap Maximum allowed gap between points 
    on the same line to link them. (pixels)"""
    mLG = 150
    im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    crop_frame_border = True
else:
    canny_thresh1, canny_thresh2 = 30, 300
    threshold = 125
    mLL = 600
    mLG = 250
    crop_frame_border = False

"""this method may downsample, so set the video writer 
resolution to the processed image resolution"""
img, edges = draw_lines(im, rho=rho, theta=theta, mll=mLL, 
                        mlg=mLG, threshold=threshold, ds=1)    
sz = (img.shape[1], img.shape[0])
print(sz)

In [ ]:
if cap.isOpened():
    fps = 20
    vout = cv2.VideoWriter()
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vout.open(file.split('.')[0] + '-quadrat-demo.mp4', fourcc, fps, sz, True)
    print('opened stream', sz)
else:
    print('cap is not open')

In [ ]:
currentFrame = 0

#for i in range(100):
while(True):
    
    # Capture frame-by-frame
    ret, im = cap.read()
    if not ret: break

    '''
    # For saving still images
    name = 'pframe_pi45_' + str(currentFrame) + '.jpg'
    save_path = os.path.join(out_path, name)
    print ('Creating...' + name)
    '''
    
    if crop_frame_border:
        im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    
    # Do processing
    img, _ = draw_lines(im, rho=rho, theta=theta, mll=mLL, 
                        mlg=mLG, threshold=threshold, ds=1, 
                        canny_1=canny_thresh1, canny_2=canny_thresh2)
    
    """Save still image in jpeg format
    cv2.imwrite(save_path, img)"""
    
    """For annotating video
    @param org Bottom-left corner of the text string.
    @param org Bottom-left corner of the text string in the image.
    @param fontFace Font type, see #HersheyFonts.
    @param fontScale Font scale factor that is multiplied by the font-specific base size.
    @param color Text color.
    @param thickness Thickness of the lines used to draw a text.
    @param lineType Line type. See #LineTypes
    """
    cv2.putText(                 # x, y
        img, str(currentFrame), (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 1, cv2.LINE_AA)
    vout.write(img)    

    # To stop duplicate images
    currentFrame += 1

# When everything done, release the capture
cap.release()
vout.release()
#cv2.destroyAllWindows()

In [ ]:
"""
# to seek into a specific frame
for _ in range(28):
    ret, im = cap.read()
"""    
"""
ret, im = cap.read()
if ret is not None:
    plt.imshow(im)
"""

In [ ]:
if crop_frame_border:
    im = im[y_trim:-y_trim, x_trim:-x_trim, :]

img, edges = draw_lines(im, rho=1, theta=np.pi/45, mll=mLL,
                        mlg=mLG, threshold=125, ds=1, 
                        canny_1=canny_thresh1, canny_2=canny_thresh2)

In [ ]:
#plt.figure(figsize=(14, 12))
plt.imshow(img)
#plt.imshow(im[y_trim:-y_trim, x_trim:-x_trim, :])

In [ ]:
plt.imshow(edges)

In [ ]:
cv2.imwrite('test.jpg', im)

In [ ]:
#cv2.circle(img, (800, 400), 10, (255, 0, 0), thickness=2, lineType=8, shift=0)
plt.figure(figsize=figsize)
plt.imshow(img)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-2.png')
plt.show()

In [ ]:
#plt.imshow(edges)

In [ ]:
#plt.imshow(img)